# Лекция 9 Введение в ООП (часть 2)

### Финансовый университет при Правительстве РФ, лектор С.В. Макрушин

v 0.7 15.02.2022

## Разделы: <a class="anchor" id="разделы"></a>
* Базовые возможности
    * [Создание классов и объектов](#создание)
    * [Наследование и полиморфизм](#наследование)
    * [Функция super()](#супер)    
    * [Проверка принадлежности к классу](#принадлежность)
    * [Базовые типы Python](#базовые-типы)                 
-

* [к оглавлению](#разделы)

In [1]:
# загружаем стиль для оформления презентации
from IPython.display import HTML
from urllib.request import urlopen
html = urlopen("file:./lec_v1.css")
HTML(html.read().decode('utf-8'))

---

## Создание классов и объектов  <a class="anchor" id="создание"></a>
* [к оглавлению](#разделы)

Создание классов

In [2]:
# Объявление класса Car, наследника класса object (общего родителя для всех классов в Python)
class Car(object):
    """Информаци о классе"""
    pass # В блоке класса объявляются функции

In [3]:
Car?

In [4]:
# Старый формат объявления класса (лучше его не использовать):
class OldCar:
    pass

В Python для именования классов принято использовать CamalCase. Методы класса, атрибуты и объекты именуются с маленькой буквы через подчеркивание.

In [5]:
class Car(object):
    """Базовый класс для автомобилей"""
    def __init__(self, x): # конструктор класса, используется для инициализации нового объекта
        self.x = x # создаем аттрибут класса        
        
    # метод класса; все методы класса должны в качестве первого атрибута иметь переменную self,
    # в которую автоматически передается ссылка на текущий объект 
    def is_near(self, x2): 
        return abs(self.x - x2) < 2.0 # self.x - обращение к атрибуту класса

Создание объектов

In [7]:
# создаем объект, при создании передаем параметр конструктора:
c_1 = Car(3.1)

In [8]:
type(c_1)

__main__.Car

In [10]:
c_1.x # получаем значение аттрибута

3.1

In [11]:
c_1.x = 11.2 # изменяем значение аттрибута
c_1.x

11.2

In [12]:
c_2 = Car(7.1)

In [13]:
c_1.is_near(c_2.x) # вызываем метод объекта

False

In [14]:
c_2.x = 10.2

In [15]:
c_1.is_near(c_2.x)

True

Фактически в Python атрибуты хранятся в словаре принадлежащем конкретному объекту:

In [16]:
c_1.__dict__

{'x': 11.2}

In [17]:
c_2.__dict__

{'x': 10.2}

In [18]:
# При желании к атрибуту объекта можно обратится так:
c_2.__dict__['x']

10.2

In [19]:
# Из-за того, что в Python атрибуты по сути это значения в словаре 
# в каждом конкретном экземпляре класса можно динамически добавлять новые атрибуты, например:
c_2.speed = 88.5

In [20]:
c_2.speed

88.5

In [21]:
c_2.__dict__

{'x': 10.2, 'speed': 88.5}

In [22]:
# При этом набор атрибутов объекта c_1 не изменился:
c_1.__dict__

{'x': 11.2}

## Наследование и полиморфизм  <a class="anchor" id="наследование"></a>
* [к оглавлению](#разделы)

In [23]:
# Класс, наследующий у Car
class CargoCar(Car): 
    def __init__(self, x, max_load, load):
        self.x = x
        self.max_load = max_load
        self.load = load
    
    def is_overloaded(self):
        return self.load > self.max_load

In [24]:
cc_1 = CargoCar(6.0, 10, 2)

In [25]:
cc_1.is_near(7.9) # метод унаследован у Car

True

In [26]:
cc_1.is_overloaded()

False

In [28]:
cars1 = [c_1, cc_1, c_2]

In [29]:
for c in cars1:
    print(c.is_near(8.2))

False
False
False


In [31]:
# Класс, наследующий у CargoCar
class CargoCarWithTrailer(CargoCar): 
    def __init__(self, x, max_load, load, trailer_length):
        super().__init__(x, max_load, load) # способ исползьовать реализацию конструктора базового класса
        self.trailer_length = trailer_length
    
    def is_near(self, x2): # перегруженный метод
        return self.x-self.trailer_length-2.0 < x2 < self.x+2.0    
    
    def is_near_old(self, x2):
        return super().is_near(x2) # способ обратиться к реализации функции в родительском классе

In [32]:
ccwt_1 = CargoCarWithTrailer(6.0, 10, 2, trailer_length=3)

In [33]:
cc_1.x, ccwt_1.x

(6.0, 6.0)

In [35]:
# демонстрация полиморфизма:
cc_1.is_near(3.0), ccwt_1.is_near(3.0)

(False, True)

Утиная типизация

In [36]:
# класс, не входящий в иерархию классов Car 
class Man(object):
    def __init__(self, name, position): 
        self.name = name
        self.position = position
    
    def is_near(self, pos2): # метод класса
        return abs(self.position - pos2) < 1.0   

In [39]:
m_1 = Man('Ivan', 9.5)

In [40]:
different_objects = [c_1, cc_1, c_2, ccwt_1, m_1]

In [41]:
# Благодаря поддержке утиной типизации в Python объекты, представляющие неродственные классы, 
# но реализующие необходимый функционал, могут обрабатываться единообразно:
for ob in different_objects:
    print(ob.is_near(6.2))

False
True
False
True
False


## Функция super()  <a class="anchor" id="супер"></a>
* [к оглавлению](#разделы)

Метод super() в методе возвращает объект который делегирует вызовы методов методам родительского класса. Наиболее наглядно его работу можно увидеть при использовании в конструкторе.

In [42]:
class Point2D(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y

class Point3D(Point2D):
    def __init__(self, x, y, z):
        super().__init__(x, y) # вызываем реализацию __init__ родительского класса
        self.z = z

In [43]:
p2d_1 = Point3D(0, -1, 1)
p2d_1.__dict__

{'x': 0, 'y': -1, 'z': 1}


## Проверка принадлежности к классу  <a class="anchor" id="принадлежность"></a>
* [к оглавлению](#разделы)

In [44]:
# Функция type() позволяет определить класс объека:
for ob in different_objects:
    print(type(ob), ob.is_near(8.2))

<class '__main__.Car'> False
<class '__main__.CargoCar'> False
<class '__main__.Car'> False
<class '__main__.CargoCarWithTrailer'> False
<class '__main__.Man'> False


In [42]:
# проверка принадлежности объека определенному классу:
type(c_1) == Car

True

In [46]:
# другой сопособ получить класс объекта:
c_1.__class__ == Car

True

In [47]:
cc_1.__class__ == Car

False

In [48]:
# чаще всего нужно знать не точную принадлежность к классу, 
# а принадлежность к классу или его наследникам 
#(так как именно это нужно для корректной работы полиморфизма):
for ob in different_objects:
    print('Класс: {}, является подклассом Car:\
    {}, проверка близости: {}'.format(type(ob), 
                                      isinstance(ob, Car), ob.is_near(8.2)))

Класс: <class '__main__.Car'>, является подклассом Car:    True, проверка близости: False
Класс: <class '__main__.CargoCar'>, является подклассом Car:    True, проверка близости: False
Класс: <class '__main__.Car'>, является подклассом Car:    True, проверка близости: False
Класс: <class '__main__.CargoCarWithTrailer'>, является подклассом Car:    True, проверка близости: False
Класс: <class '__main__.Man'>, является подклассом Car:    False, проверка близости: False


In [50]:
# Можно проверить, является ли один класс потомком другого:
issubclass(CargoCar, Car), issubclass(Car, CargoCar), issubclass(Car, Man),\
issubclass(Man, Car)

(True, False, False, False)


## Базовые типы Python  <a class="anchor" id="базовые-типы"></a>
* [к оглавлению](#разделы)

Базовые типы Python являются объектами.

In [51]:
s1 = 'abc'

In [52]:
isinstance(s1, str)

True

In [49]:
issubclass(str, object)

True

In [65]:
s1.index('b') # вызов метода объекта.

1

In [50]:
len(s1)

3